In [83]:
import pandas as pd
import cplex
from cplex.exceptions import CplexError
from collections import Iterable

In [101]:
# Reading the data
#CAC40 = pd.read_excel('./data_if/CAC40.xlsx', index_col = 'Date')
#dowjones = pd.read_excel('./data_if/dowjones.xlsx', index_col = 'Date')
CAC40 = pd.read_excel('./data_if/CAC40.xlsx').drop(columns = ['Date'])
dowjones = pd.read_excel('./data_if/dowjones.xlsx').drop(columns = ['Date'])

In [102]:
# Compute returns
def compute_returns(df):
    returns = pd.DataFrame(data = [])
    l = len(df)
    for asset in df:
        for t in range(1, l):
            returns.at[t - 1, asset] = (df.at[t, asset] - df.at[t - 1, asset]) / df.at[t - 1, asset]
    return returns

In [103]:
# Covariance matrices
CAC40 = compute_returns(CAC40)
dowjones = compute_returns(dowjones)
CAC40_cov = CAC40.cov()
dowjones_cov = dowjones.cov()
#CAC40.head()

,AC,ACA,AI,AIR,BN,BNP,CA,CAP,CS,DG,...,RI,RNO,SAF,SAN,SGO,SU,TEC,UG,VIE,VIV
0,0.001459,-0.001311,-0.009770,-0.018567,0.004193,-0.012512,0.019758,-0.027008,0.002191,0.011883,...,-0.018415,-0.000777,-0.012683,0.010626,0.000211,0.004270,-0.012945,0.008165,-0.013033,-0.009258
1,0.005558,0.010032,0.015175,-0.008502,0.001386,-0.008145,0.011421,0.014446,-0.003825,0.031616,...,0.005628,0.000778,0.000000,0.006129,0.016013,0.016156,0.008197,0.012879,0.026826,-0.006087
2,0.010469,0.014258,0.001496,0.029529,-0.004857,0.001825,0.003932,0.026193,0.021396,0.000000,...,-0.011192,0.008672,0.000642,0.006984,0.001215,0.000000,0.044716,0.017839,0.000000,0.004140
3,-0.004321,-0.011496,0.003731,0.014837,-0.014643,-0.004555,-0.006864,0.008624,0.006980,-0.001751,...,0.005659,0.026563,-0.004493,0.000000,-0.020238,-0.013387,-0.007782,-0.004836,-0.019000,0.000344
4,-0.006353,-0.006900,0.000744,0.016849,0.002122,0.000916,-0.004161,-0.006290,-0.001599,-0.002631,...,0.009380,0.021313,0.007093,-0.005215,-0.004379,0.000851,0.011765,0.010326,-0.006200,-0.000773


In [92]:
# Correlation matices
CAC40_cor = CAC40.corr()
dowjones_cor = dowjones.corr()

In [93]:
def flatten(iterable):
    for el in iterable:
        if isinstance(el, Iterable) and not isinstance(el, str): 
            yield from flatten(el)
        else:
            yield el

In [94]:
#dim = CAC40_cor.shape[0]
#X_cac40 = [["x_{}_{}".format(i, j) for j in range(1, dim + 1)] for i in range(1, dim + 1)]
#X_cac40 = flatten(X_cac40)
#list(flatten(X_cac40))
#CAC40_cor.values.flatten().tolist()

In [95]:
[["x", "y"] , [1]*2]

[['x', 'y'], [1, 1]]

In [104]:
def setProblemData(p, index_cor, q):
    
    dim = index_cor.shape[0]
    p.objective.set_sense(p.objective.sense.maximize)
    # define colnames
    X = [["x_{}_{}".format(i, j) for j in range(1, dim + 1)] for i in range(1, dim + 1)]
    Y = ["y_{}".format(i) for i in range(1, dim + 1)]
    
    cor_values = index_cor.values.flatten().tolist()
    # add x_i_j
    my_ub = [1] * dim * dim
    my_lb = [0] * dim * dim
    p.variables.add(obj = cor_values, types = [p.variables.type.integer] * (dim * dim), names = list(flatten(X)),
                    ub = my_ub, lb = my_lb)
    # add y_i_j
    my_ub = [1] * dim
    my_lb = [0] * dim
    p.variables.add(obj = [0] * dim, types = [p.variables.type.integer] * dim, names = Y, ub = my_ub, lb = my_lb)
    # add q constraint
    senses = "E"
    rhs = [q]
    rows = [[Y, [1] * dim]]
    for i in range(dim):
        rows.append([X[i], [1] * dim])
        senses += "E"
        rhs.append(1)
    
    for i in range(dim):
        for j in range(dim):
            rows.append([[X[i][j], Y[j]], [1, -1]])
            senses += "L"
            rhs.append(0)
    #print(rows)
    
    p.linear_constraints.add(lin_expr = rows, senses = senses, rhs = rhs)
    
    # p.linear_constraints.add(lin_expr = [[Y, [1] * dim]], senses = "E", rhs = [q], names = ["c0"])
    """
    for i in range(dim):
        p.linear_constraints.add(lin_expr = [[X[i], [1] * dim]], senses = "E", rhs = [1], names = ["c{}".format(str(i+1))])
        
        for j in range(dim):
            p.linear_constraints.add(lin_expr = [[[X[i][j], Y[i]], [1, -1]]], senses = "L", rhs = [0], 
                                    names = ["c{}".format(str(dim + i + j + 2))])
    """
    

In [105]:
try:
    p = cplex.Cplex()
    setProblemData(p, CAC40_cor, 10)
    p.solve()
    numrows = p.linear_constraints.get_num()
    numcols = p.variables.get_num()
    
    print(numrows)
    print()
    
    print("solution status : {} : {}".format(p.solution.get_status(), 
                                             p.solution.status[p.solution.get_status()]))
    print("solution value : {}".format(p.solution.get_objective_value()))
    slack = p.solution.get_linear_slacks()
    pi    = p.solution.get_dual_values()
    x     = p.solution.get_values()
    dj = p.solution.get_reduced_costs()
    
    for i in range(numrows):
        print ("Row %d:  Slack = %10f  Pi = %10f" % (i, slack[i], pi[i]))
    for j in range(numcols):
        print ("Column %d:  Value = %10f Reduced cost = %10f" % (j, x[j], dj[j]))
    
except CplexError as exc:
    print (exc)

CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
Reduced MIP has 931 rows, 930 columns, and 2730 nonzeros.
Reduced MIP has 930 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.05 sec. (2.63 ticks)
Found incumbent of value 14.867027 after 0.07 sec. (3.17 ticks)
Probing time = 0.05 sec. (1.36 ticks)
Tried aggregator 1 time.
Reduced MIP has 931 rows, 930 columns, and 2730 nonzeros.
Reduced MIP has 930 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.03 sec. (2.99 ticks)
Probing time = 0.05 sec. (1.36 ticks)
Clique table members: 930.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.04 sec. (2.42 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap

*     0+    0                           14.8670      416.4382              --- 
*   

CPLEX Error  1017: Not available for mixed-integer problems.


solution status : 101 : MIP_optimal
solution value : 23.02020612012774
CPLEX Error  1017: Not available for mixed-integer problems.
